<a href="https://colab.research.google.com/github/vg11072001/natural-language-processing/blob/main/deep%20learning%20-%20RNN%2C%20LSTM%2C%20BERT%2C%20Transformer/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [30]:
import pandas as pd

In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
cd /content/gdrive/MyDrive/train.csv

/content/gdrive/MyDrive/train.csv


In [33]:
df=pd.read_csv('train.csv')

In [34]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [35]:
df.shape

(20800, 5)

In [36]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [37]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [38]:
###Drop Nan Values
df=df.dropna()


In [39]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [40]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [41]:
## Get the Dependent features
y=df['label']

In [42]:
X.shape

(18285, 4)

In [43]:
y.shape

(18285,)

In [44]:
import tensorflow as tf

In [45]:
tf.__version__

'2.9.2'

In [46]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [47]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [48]:
messages=X.copy()

In [49]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [50]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [51]:
messages.reset_index(inplace=True)

In [52]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [53]:
import nltk
import re
from nltk.corpus import stopwords

In [54]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [56]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [57]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [58]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4356, 4710, 733, 3657, 2676, 4649, 209, 1439, 3858, 2234],
 [391, 666, 424, 1797, 2180, 247, 3727],
 [1876, 4312, 172, 4363],
 [1544, 1741, 4658, 696, 4402, 4151],
 [1590, 2180, 3266, 3861, 4167, 4764, 2180, 113, 3227, 2102],
 [2158,
  3225,
  710,
  542,
  1387,
  337,
  3748,
  4847,
  1319,
  1214,
  2660,
  2628,
  2123,
  3835,
  3727],
 [119, 584, 4314, 1443, 2690, 782, 1843, 135, 30, 4054, 84],
 [541, 3250, 251, 1978, 3279, 2698, 337, 1995, 30, 4054, 84],
 [350, 4742, 342, 4826, 1808, 1651, 996, 2464, 337, 3037],
 [1929, 2017, 3687, 3933, 282, 1339, 2831, 2036],
 [1280, 31, 2460, 2732, 1536, 2293, 3836, 4395, 4880, 4702, 3606],
 [696, 312, 2676, 1651, 337, 3279],
 [1866, 2804, 1598, 2497, 4893, 102, 4424, 4338, 4311],
 [1755, 3997, 151, 3364, 358, 4824, 2592, 30, 4054, 84],
 [2529, 3945, 529, 4210, 2329, 30, 4054, 84],
 [2822, 4972, 1465, 883, 1032, 117, 2137, 560, 1785, 3232],
 [4013, 1969, 666],
 [3272, 3544, 453, 2658, 337, 2934, 1818, 3727],
 [2547, 1954, 424, 2886, 3119, 

In [59]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [60]:
onehot_repr[1]

[391, 666, 424, 1797, 2180, 247, 3727]

### Embedding Representation

In [61]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4356 4710  733 ...    0    0    0]
 [ 391  666  424 ...    0    0    0]
 [1876 4312  172 ...    0    0    0]
 ...
 [4007  940 2141 ...    0    0    0]
 [4220 3279  931 ...    0    0    0]
 [ 703 4849 4358 ...    0    0    0]]


In [62]:
embedded_docs[1]

array([ 391,  666,  424, 1797, 2180,  247, 3727,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [63]:
embedded_docs[0]

array([4356, 4710,  733, 3657, 2676, 4649,  209, 1439, 3858, 2234,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [64]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
len(embedded_docs),y.shape

(18285, (18285,))

In [66]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [67]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [69]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 10ms/step - loss: 0.3252 - accuracy: 0.8389 - val_loss: 0.2026 - val_accuracy: 0.9180
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1399 - accuracy: 0.9479 - val_loss: 0.2170 - val_accuracy: 0.9148
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0943 - accuracy: 0.9673 - val_loss: 0.2197 - val_accuracy: 0.9089
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0653 - accuracy: 0.9793 - val_loss: 0.2723 - val_accuracy: 0.9137
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0422 - accuracy: 0.9874 - val_loss: 0.3220 - val_accuracy: 0.9092
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0287 - accuracy: 0.9919 - val_loss: 0.3817 - val_accuracy: 0.9094
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0228 - accuracy: 0.9925 - val_loss: 0.3955 - val_accuracy: 0.9090
Epoc

### Adding Dropout 

In [70]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [71]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [72]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [73]:
from sklearn.metrics import confusion_matrix

In [74]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [76]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
